# Reporte Técnico

En este reporte se detallará el funcionamiento del código realizado para el esquema de firma digital implementado en la solución de este reto. 

# Librerías y archivos

Es importante destacar que esta aplicación web está construida a través de Python, por lo que para el correcto funcionamiento de esta, es necesario contar con las librerías **flask**, **flask-login**, **flask-sqlalchemy**, **hashlib**, **json**, **binascii**, **sqlite3**, **pycryptodome** y **werkzeug**. También, se debe contar con las carpetas para los templates .html y los recursos de estas en el mismo directorio que los archivos .py.

# Python

Primeramente, se cuenta con un archivo .py:


*   **App.py**. En este archivo se encuentra el funcionamiento principal de la aplicación. Es el encargado de hechar a andar el servidor y de crear las funciones y rutas para realizar las operaciones deseadas. 





# forms.py

Este archivo se puede implementar para definir las clases correspondientes de la información que recibirán los formularios que se manejarán en la aplicación, como el de inicio de sesión, registro y firma/verificación, si así se desea, pero la información se puede manejar directamente desde los métodos request de Flask.

In [ ]:
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField, PasswordField, BooleanField
from wtforms.validators import DataRequired, Length, InputRequired, EqualTo

#### Formularios para el registro de usuarios ####
class SignupForm(FlaskForm):
    usuario = StringField('Usuario', validators=[DataRequired(), Length(max=64)])
    contraseña = PasswordField('Contraseña', validators=[InputRequired(), EqualTo('confirmar',message='Las contraseñas deben coincidir')])
    confirmar = PasswordField('Confirmar contraseña') 
    ID = StringField('ID', validators=[DataRequired(),  Length(min=11,max=11)])
    departamento = StringField('Departamento', validators=[DataRequired(), Length(max=64)])
    submit = SubmitField('Registrar')

#### Formularios para el inicio de sesión ####
class LoginForm(FlaskForm):
    ID = StringField('ID:', validators=[DataRequired(), Length(min=11,max=11)])
    contraseña  = PasswordField('Contraseña:', validators=[DataRequired()])
    remember_me = BooleanField('Recuérdame')
    entrar = SubmitField('Entrar')# -*- coding: utf-8 -*-

#### Formularios para le firma/verificación ####
class SignatureForm(FlaskForm):
    contraseña  = PasswordField('Contraseña:', validators=[DataRequired()])
    subir = SubmitField('Subir')

# App.py

Primeramente se cargan los paquetes necesarios de cada librería.

In [ ]:
from flask import Flask, render_template, request, redirect, url_for, flash, session
from flask_login import LoginManager, login_user, current_user, logout_user, login_required
from flask_sqlalchemy import SQLAlchemy

from flask_login import UserMixin
from werkzeug.security import generate_password_hash, check_password_hash

from forms import SignupForm, LoginForm, SignatureForm
from werkzeug.urls import url_parse
from werkzeug.utils import secure_filename

import hashlib
import json
import binascii
import sqlite3

from Crypto.PublicKey import RSA

Después, se configuran los parámetros de la aplicación a través de flask.

In [ ]:
app = Flask(__name__)
app.config['SECRET_KEY'] = '!b8^cw1u#6gg)=yj7=x1(^8(z4-9holmnsz%alvf%jd^2&v4d=' #Llave secreta para la sesión
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database2.db' #Conexión con la base de datos. 
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

login_manager = LoginManager(app)
login_manager.login_view = "login" #Configuración para el inicio de sesión

db = SQLAlchemy(app) # Instancia a la que se llamará al crear la clase para el usuario y el tipo de información que contiene

Cabe destacar que en este ejemplo se está haciendo una conexión a una base de datos sqlite hosteada localmente, pero SQLAlchemy puede hacer la conexión a cualquier base de datos (MySQL, Postres, Oracle, etc.). A continuación, se define la clase para el usuario, los atributos que cada instancia contenerá y los métodos que puede usar.

In [ ]:
class User(db.Model, UserMixin):
    __tablename__ = 'usuarios_sistema' #nombre de la tabla en la base de datos

    ################ Atributos de la clase  ################
    id = db.Column(db.Integer, primary_key=True)
    usuario = db.Column(db.String(80), nullable=False)
    id_per = db.Column(db.String(256), unique=True, nullable=False)
    contraseña = db.Column(db.String(128), nullable=False)
    departamento = db.Column(db.String(80), nullable=False)
    n = db.Column(db.String, nullable=False)
    e_public = db.Column(db.String, nullable=False)
    d_priv = db.Column(db.String, nullable=False) 
    
    def __repr__(self):
        return f'<User {self.id_per}>'
    
    ################ Guarda la contraseña hasheada para el usuario  ################
    def set_password(self, contraseña):
        self.contraseña = generate_password_hash(contraseña)
        
    ################ Revisa si la contraseña recibida y la del usuario coinciden  ################
    def check_password(self, contraseña):
        return check_password_hash(self.contraseña, contraseña)
    
    ##### Guardar los datos de cada usuario en la base de datos #####
    def save(self):
        if not self.id:
            db.session.add(self)
        db.session.commit()
    
    #### Recupera el id asignado a cada usuario dentro de la base de datos ####
    @staticmethod
    def get_by_id(id):
        return User.query.get(id)
    
    #### Recupera el id personal del usuario (Número de seguro social) ####
    @staticmethod
    def get_by_ID(id_per):
        return User.query.filter_by(id_per=id_per).first()
    
    #### Recupera el usuario (username) del usuario ####
    @staticmethod
    def get_by_usuario(usuario):
        return User.query.filter_by(usuario=usuario).first()

db.create_all() #Genera la tabla bajo el nombre usuarios_sistema con las columnas especificadas

Una vez se tiene definida la clase para cada usuario, definimos una función que nos ayudará a conectarnos a la base de datos para hacer los querys necesarios. Además, se crea otra función para para cargar al usuario y poder manejar los inicios de sesión.

In [ ]:
def get_db_connection():
    conn = sqlite3.connect('database2.db')
    conn.row_factory = sqlite3.Row
    return conn

@login_manager.user_loader
def load_user(user_id):
    return User.get_by_id(int(user_id))

A partir de este punto, se empiezan a generar las vistas y rutas para cada acción dentro de la aplicación. Para crear una nueva ruta, se utiliza 

```
@app.route()
```
que recibe como parámetros el url deseado y los métodos que esta contendrá. Además, después de cada ruta se define una función que contendrá el funcionamiento de dicha ruta.


De esta forma, creamos la ruta principal de nuestra aplicación, que renderiza la información y los campos requeridos para hacer el inicio de sesión contenidos en index.html. Para esta vista, en caso de que haya una sesión activa se redirecciona inmediatamente a la página con el menu para realizar las operaciones deseadas. En caso de que el usuario haya intentado acceder a una vista protegida sin tener una sesión, se le redirecciona a esta pagina. Si no es ninguno de los casos anteriores, una vez se valida la información de inicio de sesión se redirecciona al menu.

In [ ]:
@app.route('/',methods=['GET', 'POST'])
def login():
    if current_user.is_authenticated:
        return redirect(url_for('menu'))
    
    if request.method == 'POST':
        user = User.get_by_ID(request.form['user'])
        if user is not None and user.check_password(request.form['password']):
            login_user(user, remember=request.form['remember'])
            next_page = request.args.get('next')
            
            if not next_page or url_parse(next_page).netloc != '':
                next_page = url_for('menu')
            return redirect(next_page)
    return render_template('index.html')

La siguiente ruta es para el caso en que no se cuente con un usuario y se desee registrar uno nuevo. Al igual que en la ruta anterior, si se cuenta con inicio de sesión activo se redirecciona inmediatamente a la pagina del menu. Esta ruta simplemente valida la información recibida y la inseerta dentro de la tabla que lleva el registro para cada usuario. Además, es en este momento en que se generar las llaves bajo el esquema de firma digital de RSA para cada usuario.

In [ ]:
@app.route("/signup/", methods=["GET", "POST"])
def show_signup_form():
    if current_user.is_authenticated:
        return redirect(url_for('firmar'))
    
    error = None
    
    if request.method == 'POST':
        usuario = request.form['user']
        ID = request.form['userID']
        departamento = request.form['dpto']
        contraseña = request.form['password']
        
        user = User.get_by_ID(ID)
        usern = User.get_by_usuario(usuario)
        
        if user is not None:
            error = f'El ID {ID} ya está siendo utilizado por otro usuario'
            flash(error)
        
        elif usern is not None:
            error = f'El nombre {usuario} ya está siendo utilizado por otro usuario'
            flash(error)
        else:
            
            
            keyPair = RSA.generate(bits=1024)
            
            user = User(usuario=usuario,
                        id_per=ID,
                        departamento=departamento,
                        n = str(keyPair.n),
                        e_public = str(keyPair.e),
                        d_priv = str(keyPair.d)
                        )
            
            user.set_password(contraseña)
            user.save()
            
            login_user(user,remember=True)
            next_page = request.args.get('next', None)
            if not next_page or url_parse(next_page).netloc != '':
                next_page = url_for('menu')
            return redirect(next_page)
    return render_template("signup.html")

A continuación se crea la ruta en la que se desplegará toda la información pertinente a las operaciones que se deseen realizar a través de renderizar el archivo menu.html. Por default, se redirecciona a la ruta */menu/*, y utilizando los botones se redirecciona a la ruta deseada para cada operación.

In [ ]:
@app.route("/menu/", methods=["GET", "POST"])
@login_required
def menu():
    return render_template('menu.html')

Una vez se selecciona la operación a realizar (firmar o verificar), se redirige a las siguientes rutas cuyo funcionamiento se explica más adelante.

In [ ]:
@app.route("/menu/firmar", methods=["GET", "POST"])
@login_required
def firmar():

    if request.method == 'POST':
        
        user = User.get_by_ID(current_user.id_per)
        
        if user.check_password(request.form['gpsswd1']):
            
            if request.method == 'POST':
                
                doc = request.files['gfile']
                ruta_archivo = './documentos_recibidos/'
                doc.save(ruta_archivo+secure_filename(doc.filename))
                
                docRead = open(ruta_archivo+secure_filename(doc.filename),'rb').read().hex()
                
                doc_hash = hashlib.sha512(str(docRead).encode('utf-8')).hexdigest()
                
                doc_int = int.from_bytes( binascii.unhexlify(doc_hash),byteorder='big')
                
                
                conn = get_db_connection()
                cursor = conn.cursor()
                data = cursor.execute('SELECT e_public, n, d_priv FROM usuarios_sistema WHERE id_per = (?)',(current_user.id_per,)).fetchall()
                
                e_public = data[0][0]
                n = data[0][1]+'A'
                d_priv = data[0][2]+'A'
                
                n_n = n[:-1]
                d_priv_n = d_priv[:-1]
                
                
                d_priv_int= int(d_priv_n)
                n_int = int(n_n)
                
                firma = hex(pow(doc_int, d_priv_int, n_int))


                doc_name = secure_filename(doc.filename)
                
                conn = get_db_connection()
                cursor = conn.cursor()
                cursor.execute('''
                               INSERT INTO firmas (document_id,
                                                   document_name,
                                                   firma,
                                                   id_per,
                                                   usuario,
                                                   e_public,
                                                   n,
                                                   d_priv
                                                   )
                               VALUES (?,?,?,?,?,?,?,?)
                               ''',(doc_hash,doc_name,firma,current_user.id_per,current_user.usuario,e_public,n,d_priv))

                conn.commit()
                conn.close()
                
                next_page = request.args.get('next')

                return redirect(url_for('exitoFirma'))
    
                if not next_page or url_parse(next_page).netloc != '':
                    next_page = url_for('exitoFirma')
                return redirect(next_page)

    return render_template(url_for('menu'))

Dentro de esta ruta primeramente se valida la contraseña del usuario activo y el archivo que se desee firmar. Una vez se cuenta con el archivo, se guarda en alguna ruta deseada para posteriormente poder leer el archivo y hashearlo. Este se firma utilizando la llave privada del usuario en sesión a través del esquema RSA, para posteriormente almacenar la firma, el hasheo del archivo y la llave pública de quien hizo la firma. 

Ahora, se puede proceder a verificar qué archivos han sido firmados por quién esto a través de la siguiente ruta definida.

In [ ]:
@app.route("/menu/verificar", methods=["GET", "POST"])
@login_required
def verificar():

    if request.method == 'POST':
        
        user = User.get_by_ID(current_user.id_per)
        
        if user.check_password(request.form['gpsswd2']):
            
            if request.method == 'POST':
                
                doc = request.files['vfile']
                ruta_archivo = './documentos_verificados/'
                doc.save(ruta_archivo+secure_filename(doc.filename))
                docRead = open(ruta_archivo+secure_filename(doc.filename),'rb').read().hex()
                
                doc_hash = hashlib.sha512(str(docRead).encode('utf-8')).hexdigest()
                
                doc_int = int.from_bytes(binascii.unhexlify(doc_hash),byteorder='big')


                conn = get_db_connection()
                cursor = conn.cursor()
                data = cursor.execute('SELECT usuario, e_public, n, firma FROM firmas WHERE document_id = (?)',(doc_hash,)).fetchall()
                conn.close()
                
                
                llaves = {}
                
                ver_binary = 0
                for fila in data:
                    e = fila[1]

                    n = fila[2]
                    n_n = n[:-1]
                
                    llaves[fila[0]] = (e,n_n,fila[3])

                    hashFromSignature = pow(int(fila[3],base=16), int(e), int(n_n))
                    if hashFromSignature == doc_int:
                        ver_binary = 1
                    else:
                        ver_binary = 0
                        
                if ver_binary == 1:
                
                    next_page = request.args.get('next')
                    
                    conn = get_db_connection()
                    cursor = conn.cursor()
                    quien_firmo = cursor.execute('SELECT usuario FROM firmas WHERE document_id = (?)',(doc_hash,)).fetchall()
                    
                    conn.close()
                    
                    usuarios_que_firmaron = []
                    for i in quien_firmo:
                        for j in i:
                           usuarios_que_firmaron.append(j)
                           
                    usuarios_que_firmaron = json.dumps(usuarios_que_firmaron)
                    session['usuarios_que_firmaron'] = usuarios_que_firmaron
                    
                    return redirect(url_for('exitoVerificacion',usuarios_que_firmaron=usuarios_que_firmaron))
        
                    if not next_page or url_parse(next_page).netloc != '':
                        next_page = url_for('exitoVerificacion')
                    return redirect(next_page)
                else:
                    
                    next_page = request.args.get('next')

                    return redirect(url_for('fracasoVerificacion'))
        
                    if not next_page or url_parse(next_page).netloc != '':
                        next_page = url_for('fracasoVerificacion')
                    return redirect(next_page)
                    

    return render_template(url_for("menu"))

Para poder verificar los archivos, primeramente se realiza el mismo proceso de lectura y hasheo de la firma para el archivo cargado por el usuario. Una vez realizado esto, se hace un query en la tabla de *firmas* con base en el hash del archivo para buscar qué usuarios han firmado ese mismo documento. Una vez se han obtenido todas las firmas para ese mismo documento, se realiza el proceso de verificación para cada una de ellas, comparando el documento hasheado con el resultado de las operaciones RSA con las llaves públicas. Si todas las firmas son correctas, entonces se redirecciona a la ruta */verificacion_EXITOSA/* en la que se despliegan los usuarios que han firmado dicho documento; en caso de que alguna firma no coincida, se redirecciona a la ruta */verificacion_FALLIDA/* que indica que el documento no está firmado o no coincide alguna firma.

Por último, cabe destacar que al igual que la ruta para firmar, esta es una vista protegida, por lo que el usuario no podrá acceder a ella a menos de que haya iniciado sesión.

Con las rutas para verificar y firmar en mente, se definieron entonces las rutas ya mencionadas que desplegarán el estado de la acción, renderizando la vista correspondiente.

In [ ]:
#### Ruta para saber que la firma fue exitosa ####
@app.route("/firma_EXITOSA/")
def exitoFirma():
    return render_template('firma_exitosa.html')

#### Ruta para saber que se verificaron las firmas de algún documento ####
@app.route("/verificacion_EXITOSA/")
def exitoVerificacion():
    usuarios_que_firmaron = request.args['usuarios_que_firmaron']
    usuarios_que_firmaron = session['usuarios_que_firmaron'] 
    return render_template('verificado.html',usuarios_que_firmaron=usuarios_que_firmaron)

#### Ruta para saber que la verificación no fue correcta ####
@app.route("/verificacion_FALLIDA/")
def fracasoVerificacion():
    return render_template('verificación_fallida.html')

Después, se creó la ruta necesaria para poder cerrar sesión, utilizando las funciones implementadas de flask. Una vez se cierra sesión, se redirecciona al usuario a la pagina de inicio de sesión.

In [ ]:
@app.route('/logout')
def logout():
    logout_user()
    return redirect(url_for('login'))

Finalmente, incluimos la inicialización del servidor a través del puerto deseado

In [ ]:
if __name__ == '__main__':
    app.run(port = 3000, debug=True)